<a href="https://colab.research.google.com/github/orizay/free-fonts/blob/master/sklearn_%EC%A0%84%EC%B2%98%EB%A6%AC_%EA%B8%B0%EB%B3%B81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# 데이터 셋: 타이타닉 데이터셋(출처:Kaggle.com)
- https://bit.ly/fc-ml-titanic

In [3]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


- PassengerId : 승객 아이디
- Survived : 생존 여부, 1:생존 ; 0: 사망
- Pclass : 등급
- Name : 성합
- Sex : 성별
- Age : 나이
- SibSp : 형제,자매,배우자 수
- Parch : 부모,자식 수
- Ticket : 티켓 번호
- Fare : 요금
- Cabin : 좌석번호
- Embarked : 탑승 항구

# 전처리 :train / validation 세트 나누기
1.먼저,feature와 label을 정의 합니다


2.feature/label을 정의했으면,적절한 비율로 train/validation set을 나눕니다

In [4]:
feature = ['Pclass','Sex','Age','Fare']

In [5]:
label = ['Survived']

In [6]:
train[feature].head()

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500


In [7]:
train[label].head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [9]:
from sklearn.model_selection import train_test_split

- **test_size:** validation set에 할당할 비율(20% -> 0.2)
- **shuffle:** 셔플 옵션(기본 True)
- **random_state:** 랜덤 시드값

# **return 받는 데이터의 순서가 중요합니다.**

In [11]:
x_train,x_valid,y_train,y_valid = train_test_split(train[feature],train[label],test_size=0.2,shuffle=True,random_state=30)

In [12]:
x_train.shape,y_train.shape

((712, 4), (712, 1))

In [13]:
x_valid.shape,y_valid.shape

((179, 4), (179, 1))

# 전처리 : 결측치

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## 결측치를 확인하는 방법은 pandas의 isnull()/그리고 합계를 구하는 sum()을 통해 한 눈에 확인할 수 있습니다.

In [15]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## 개별 column의 결측치에 대하여 확인하는 방법은 다음과 같습니다.

In [16]:
train['Age'].isnull().sum()

177

# **Impute 도큐먼드**

## 1.수치형(Numerical Column)데이터에 대한 결측치 처리

In [17]:
train['Age'].fillna(0).describe()

count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [18]:
train['Age'].fillna(train['Age'].mean()).describe()

count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: Age, dtype: float64

In [19]:
from sklearn.impute import SimpleImputer

In [20]:
imputer = SimpleImputer(strategy='mean')

## fit()을 통해 결측치에 대한 학습을 진행합니다

In [21]:
imputer.fit(train[['Age','Pclass']])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

## transform()은 실제 결측치에 대한 처리를 해주는 함수입니다

In [22]:
result = imputer.transform(train[['Age','Pclass']])
result

array([[22.        ,  3.        ],
       [38.        ,  1.        ],
       [26.        ,  3.        ],
       ...,
       [29.69911765,  3.        ],
       [26.        ,  1.        ],
       [32.        ,  3.        ]])

In [23]:
train[['Age','Pclass']] = result

In [24]:
train[['Age','Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [25]:
train[['Age','Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.699118,2.308642
std,13.002015,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,29.699118,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


# **fit_transform()은 fit()과transform()을 한번에 해주는 함수 입니다.**

In [26]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

In [27]:
train[['Age','Pclass']].isnull().sum()

Age       177
Pclass      0
dtype: int64

In [28]:
imputer = SimpleImputer(strategy='median')

In [29]:
result = imputer.fit_transform(train[['Age','Pclass']])
train[['Age','Pclass']] = result

In [30]:
train[['Age','Pclass']].isnull().sum()

Age       0
Pclass    0
dtype: int64

In [31]:
train[['Age','Pclass']].describe()

,Age,Pclass
count,891.000000,891.000000
mean,29.361582,2.308642
std,13.019697,0.836071
min,0.420000,1.000000
25%,22.000000,2.000000
50%,28.000000,3.000000
75%,35.000000,3.000000
max,80.000000,3.000000


# 2.(Categorical Column)데이터에 대한 결측치 처리

In [32]:
train = pd.read_csv('https://bit.ly/fc-ml-titanic')

## 1개의 column을 처리하는 경우

In [33]:
train['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

## Imputer를 사용하는 경우 : 2개 이상의 column을 처리할 때

In [34]:
imputer = SimpleImputer(strategy='most_frequent')

In [35]:
result = imputer.fit_transform(train[['Embarked','Cabin']])
train[['Embarked','Cabin']] = result

In [36]:
train[['Embarked','Cabin']].isnull().sum()

Embarked    0
Cabin       0
dtype: int64